## Week 3 - Image Recognition

### Heather Tweedie, 1/2/23

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import math

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

import matplotlib.style #Some style nonsense
import matplotlib as mpl #Some more style nonsense

#Set default figure size
#mpl.rcParams['figure.figsize'] = [12.0, 8.0] #Inches... of course it is inches
mpl.rcParams["legend.frameon"] = False
mpl.rcParams['figure.dpi']=200 # dots per inch


2023-02-01 10:57:33.204879: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-01 10:57:33.482596: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-01 10:57:33.540481: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-01 10:57:33.540527: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [3]:
# load data
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# re-scale inputs
train_images=train_images/255.0
test_images=test_images/255.0

# check shape of datasets
print("Shape of training images:",train_images.shape)
print("Length of training set labels:",len(train_labels))
print("First label:",train_labels[0])
print("Shape of testing images:",test_images.shape)
print("Length of testing set labels:",len(test_labels))

Shape of training images: (60000, 28, 28)
Length of training set labels: 60000
First label: 5
Shape of testing images: (10000, 28, 28)
Length of testing set labels: 10000


In [14]:
def addNoise(image, y_noise):
    """
    Adds a random noise contribution drawn from a uniform distribution between 0 and a user-defined maximum to an image.
    
    Args:
        image: the image to which the noise will be added
        y_noise: the maximum value for the uniform distribution from which the random noise contribution will be drawn

    Returns:
        newImage: the new image with noise added
    """
    # get image dimensions
    image_x = 28
    image_y = 28

    newImage = np.empty([image_x, image_y])
    for i in range(image_x):
        for j in range(image_y):
            newImage[i,j] = image[i,j] + np.random.uniform(0, y_noise)

    return newImage
    


In [4]:
def trainWithNoise(model, image_dim, y_noise, batchSize, nepochs):

    imagesWithNoise = np.empty([10000, image_dim, image_dim])
    for i in range(len(imagesWithNoise[:,0,0])):
        imagesWithNoise[i,:,:] = addNoise(test_images[i,:,:], 3)

    history = model.fit(imagesWithNoise, train_labels, batch_size = batchSize, epochs = nepochs)

    test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
    return test_loss, test_acc

In [ ]:
def accuracyWithNoise(model, image_dim, y_noise):

    imagesWithNoise = np.empty([10000, image_dim, image_dim])
    for i in range(len(imagesWithNoise[:,0,0])):
        imagesWithNoise[i,:,:] = addNoise(test_images[i,:,:], 3)

    test_loss, test_acc = model.evaluate(imagesWithNoise, test_labels, verbose=2)
    return test_loss, test_acc

We now define an compile the network on which we will train the data...

In [9]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dense(10)
])

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=tf.keras.optimizers.SGD(lr=1.0),
              metrics=['accuracy'])

2023-02-01 10:47:49.817292: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-01 10:47:49.819360: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-01 10:47:49.819467: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (penguin): /proc/driver/nvidia/version does not exist
2023-02-01 10:47:49.822030: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Train the model on the MNIST dataset:

In [ ]:
history = model.fit(train_images, train_labels, batch_size=120, epochs=30)

Evaluate accuracy of model for different noise levels:

In [21]:
imagesWithNoise = np.empty([10000, 28, 28])
len(imagesWithNoise[:,0,0])

10000

In [22]:
image_x = 28
image_y = 28
noises = 3

imagesWithNoise = np.empty([10000, image_x, image_y])
for i in range(len(imagesWithNoise[:,0,0])):
    imagesWithNoise[i,:,:] = addNoise(test_images[i,:,:], 3)

In [ ]:
noises = np.linspace(0, 3, 2)
accuracies = np.empty([1, len(noises)])

# batchsize = 120, epochs = 30

for i in range(len(noises)):
    test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
    accuracies[i] = test_acc
